## Consist Quickstart

This notebook demonstrates Consist's core feature: intelligent caching based on code, configuration, and input data.

By the end, you'll understand:
- How Consist tracks provenance (what code/config/inputs produced each result)
- How to skip redundant computation with caching
- How to query and inspect your runs

## Installation

If Consist is not installed yet, install the package in your environment:

```bash
pip install consist
```

We'll keep everything in the current working directory so you can see the artifacts on disk.

In [1]:
from consist import Tracker
from pathlib import Path
import pandas as pd

# Clean slate for reproducible notebook output
db_path = Path("./provenance.duckdb")
if db_path.exists():
    db_path.unlink()

# Initialize Tracker:
# - run_dir: directory where outputs are stored
# - db_path: DuckDB file that stores run metadata and provenance
tracker = Tracker(run_dir="./runs", db_path="./provenance.duckdb")

## Create a small input file

We create a tiny CSV file to use as input. When you pass a file path in `inputs={...}`, Consist computes its content hash. This hash becomes part of the cache signature: if the same input file is used again with the same config, Consist recognizes it and returns cached results.

In [2]:
raw_path = Path("raw.csv")
df = pd.DataFrame({"value": [0.1, 0.6, 0.9, 0.2]})
df.to_csv(raw_path, index=False)
raw_path

PosixPath('raw.csv')

## Define a step

Define a function that accepts data and writes output. Consist will wrap this function to track its provenance.

When you return a file path, Consist recognizes it as an artifact (a tracked output) and records which run created it, with what code and config.

In [3]:
def clean_data(raw_df, threshold=0.5, out_name="cleaned.parquet"):
    cleaned = raw_df[raw_df["value"] > threshold]
    out_path = tracker.run_dir / out_name
    cleaned.to_parquet(out_path)
    return out_path

## Run it (cache miss)

Call `tracker.run()` to execute the function and record provenance.

The key parameters:
- `fn`: Your function to execute
- `inputs`: Files to hash (become part of cache signature)
- `config`: Parameters to hash (become part of cache signature)
- `runtime_kwargs`: Arguments passed to your function (do NOT affect caching)
- `outputs`: Names for your function's return values

On this first run, Consist will execute the function and record a new run.

<details>
<summary>Aside: default-tracker style (optional)</summary>

If you prefer not to pass a `Tracker` around, you can set a scoped default tracker
and call the top-level API:

```python
import consist
from consist import use_tracker

with use_tracker(tracker):
    result1 = consist.run(
        fn=clean_data,
        inputs={"raw_df": raw_path},
        config={"threshold": 0.5},
        outputs=["cleaned"],
    )
```

This can make it easier to call Consist from helper functions without threading
the tracker through every signature.
</details>


In [4]:
result1 = tracker.run(
    fn=clean_data,
    inputs={"raw_df": raw_path},
    config={"threshold": 0.5},
    runtime_kwargs={"out_name": "cleaned_0_5.parquet"},
    outputs=["cleaned"],
)
print(f"Cache hit: {result1.cache_hit}")

Cache hit: False


In [5]:
cleaned_artifact = result1.outputs["cleaned"]
print(f"Artifact: {cleaned_artifact.id}")
print(f"Path: {cleaned_artifact.path.name}")

Artifact: 56e7eff6-f229-4305-b815-9156ddab0286
Path: cleaned_0_5.parquet


## Run it again (cache hit)

Now run with identical inputs and config.

Consist computes a fingerprint (signature) from:
1. Your function's code
2. The `config` dict
3. The hashes of `inputs` files

Since we're using the same code, config, and input file, the signature matches a previous run. Instead of re-executing, Consist returns the cached result instantly.

In [6]:
result2 = tracker.run(
    fn=clean_data,
    inputs={"raw_df": raw_path},
    config={"threshold": 0.5},
    outputs=["cleaned"],
)
print(f"Cache hit: {result2.cache_hit}")

Cache hit: True


## Change config (cache miss)

Now change `config` to use a different threshold (0.8 instead of 0.5).

This changes the signature, so Consist must re-execute the function and record a new run.

Note: We use `runtime_kwargs={"out_name": "cleaned_0_8.parquet"}` to avoid overwriting the previous output. Since `runtime_kwargs` does NOT affect the signature, this doesn't cause a cache miss—only `config` and `inputs` matter for caching.

In [7]:
result3 = tracker.run(
    fn=clean_data,
    inputs={"raw_df": raw_path},
    config={"threshold": 0.8},
    runtime_kwargs={"out_name": "cleaned_0_8.parquet"},
    outputs=["cleaned"],
)
print(f"Cache hit: {result3.cache_hit}")

Cache hit: False


## Inspect recorded runs

Query recent runs by model name (derived from function name). Notice the `cache_hit` flag shows which runs were executed vs. returned from cache.

To filter runs by parameter values, see [Concepts: config vs facet](../docs/concepts.md#the-config-vs-facet-distinction).

In [8]:
runs = tracker.find_runs(model="clean_data", limit=10)
[(run.id, run.status, run.meta.get("cache_hit", False)) for run in runs]

[('clean_data_a9d6fbf0', 'completed', False),
 ('clean_data_7a0bdf8e', 'completed', True),
 ('clean_data_7857acb4', 'completed', False)]

## See the outputs on disk

Consist stores artifacts under the run directory. You can access the files directly, or load them via the Artifact object. Notice we have two .parquet files because we ran with different config values.

In [9]:
print("Output files:")
for p in sorted(Path("runs").glob("*.parquet")):
    print(f"  {p.name}")

print("\nCleaned data (threshold=0.5):")
pd.read_parquet("runs/cleaned_0_5.parquet")

Output files:
  cleaned_0_5.parquet
  cleaned_0_8.parquet

Cleaned data (threshold=0.5):


,value
1,0.6
2,0.9
